In [493]:
# Combine two dataframes into one common dataframe,
# and dump to SQL file for upload to Web app
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For munging
import re
import json

In [494]:
yalePath = os.path.join(PROJ_ROOT, "data", "interim", "yale_bios_processed.csv")
harvardPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_bios_processed.p")

yale = pd.read_csv(yalePath)
harvard = pd.read_pickle(harvardPath)

In [495]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio', u'headers'],
      dtype='object')

In [496]:
harvard.shape

(4270, 19)

In [497]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio', u'headers'],
      dtype='object')

In [498]:
harvard.columns

Index([       u'Name',         u'B/T',        u'City',       u'Event',
       u'High School',    u'Hometown',         u'Ht.',   u'Name_link',
               u'No.',    u'Position',      u'Region',         u'S/C',
            u'Weapon',         u'Wt.',         u'Yr.',      u'season',
             u'sport',     u'headers',         u'Bio'],
      dtype='object')

In [499]:
yale["headers"][0]

'{"Position:": "G", "Year:": "So.", "Height:": "5-7", "Hometown:": "Fayetteville, NY", "Weight:": "170", "High School:": "Jamesville-Dewitt", "name": "Josh Greenberg"}'

In [500]:
harvard.head()

,Name,B/T,City,Event,High School,Hometown,Ht.,Name_link,No.,Position,Region,S/C,Weapon,Wt.,Yr.,season,sport,headers,Bio
0,A.J. Jaffe,None,St. Charles,None,Marmion Academy,"St. Charles, Ill.",None,/sports/wrest/2016-17/bios/jaffe_aj,None,141,Ill.,None,None,None,Freshman,2016-17,wrest,"{""Hometown:"": ""St. Charles, Ill."", ""Position:""...",\rBefore Harvard Four-time letterwinner at Mar...
1,A.J. Kennedy,None,Fonthill,None,Notre Dame College,"Fonthill, Ont.",6-3,/sports/fball/2003-04/bios/a.j._kennedy,98,DE,Ont.,None,None,220,Freshman,2003-04,fball,"{""Position:"": ""DE"", ""Year:"": ""Freshman"", ""Heig...",
2,AJ Carvalho,None,Swansea,None,Joseph Case,"Swansea, Mass.",None,/sports/mcrew-lw/2010-11/bios/varsity/Carvalho...,None,None,Mass.,None,None,None,Sophomore,2010-13,mcrew-lw,"{""name"": ""Antonio Carvalho"", ""Year:"": ""Sophomo...",\rSophomore • 2011\r\rDid not see spring race ...
3,Aaki Vora,None,Mumbai,Freestyle,The Cathedral and John Connon School,"Mumbai, India",None,/sports/wswimdive/2016-17/bios/Vora_Aaki,None,None,India,None,None,None,Freshman,2016-17,wswimdive,"{""Hometown:"": ""Mumbai, India"", ""Position:"": ""F...",\rBefore Harvard\r\r Best female athlete at th...
4,Aaron Byrd,None,Abilene,None,Cooper,"Abilene, Texas",6-2,/sports/fball/2001-02/bios/aaron_byrd,3,DB,Texas,None,None,190,Freshman,2001-03,fball,"{""Position:"": ""DB"", ""Year:"": ""Freshman"", ""Heig...",


Definitive list of columns:
1. Name
2. B/T
3. City
4. Events
5. High School
6. Hometown
7. Height
8. No.
9. Position
10. Region
11. S/C
12. Weapon
13. Weight
14. Year
15. Active Seasons
16. Sport
17. Headers
18. Bio

Next step involves adding blank data to tables when either does not contain the above columns, and cleaning up some of the data to ensure consistency.

In [501]:
yale.columns

Index([u'Name', u'B/T', u'City', u'Cl.', u'Events', u'High School',
       u'Hometown/High School', u'Hometown/Region', u'Ht.', u'Name_link',
       u'No.', u'Pos.', u'Region', u'Weapon', u'Wt.', u'season', u'sport',
       u'Bio', u'headers'],
      dtype='object')

In [502]:
# Insert missing columns
yale["S/C"] = None
yale["College"] = "Yale"
# Re-order, then rename
# Yale
yaleColumns = ["Name", "B/T", "City", "Events", "High School", "Hometown/High School",
               "Ht.", "No.", "Pos.", "Region", "S/C", "Weapon", "Wt.", "Cl.", "season", "sport", "headers", "Bio", "College"]

In [503]:
yale = yale[yaleColumns]

In [504]:
# STILL WORKING

In [505]:
harvard.columns

Index([       u'Name',         u'B/T',        u'City',       u'Event',
       u'High School',    u'Hometown',         u'Ht.',   u'Name_link',
               u'No.',    u'Position',      u'Region',         u'S/C',
            u'Weapon',         u'Wt.',         u'Yr.',      u'season',
             u'sport',     u'headers',         u'Bio'],
      dtype='object')

In [506]:
harvard["College"] = "Harvard"
harvardColumns = ["Name", "B/T", "City", "Event", "High School", "Hometown",
               "Ht.", "No.", "Position", "Region", "S/C", "Weapon", "Wt.", "Yr.", "season", "sport", "headers", "Bio", "College"]

In [507]:
harvard = harvard[harvardColumns]

In [508]:
harvard.shape

(4270, 19)

In [509]:
yale.shape

(3004, 19)

In [510]:
# Normalize column names
normColumns = ["Name", "B/T", "City", "Events", "High School", "Hometown",
               "Ht.", "No.", "Position", "Region", "S/C", "Weapon", "Wt.", 
               "Class", "Active Seasons", "Sport", "Misc", "Bio", "College"]
yale.columns = normColumns
harvard.columns = normColumns

In [511]:
yale["Hometown"] = yale['Hometown'].apply(lambda x: x.split("/")[0].strip() if type(x) == str else "")

In [512]:
yale["Misc"][0]

'{"Position:": "G", "Year:": "So.", "Height:": "5-7", "Hometown:": "Fayetteville, NY", "Weight:": "170", "High School:": "Jamesville-Dewitt", "name": "Josh Greenberg"}'

In [513]:
stacked = pd.concat([yale, harvard], axis=0)

In [514]:
# Drop some of the columns we don't need
finalColumns = ["Name", "City", "High School", "Hometown",
                "Ht.", "No.", "Position", "Region", "Wt.",
                "Active Seasons", "Misc", "Bio", "College"]
stacked = stacked[finalColumns]

In [515]:
stacked.head()

,Name,City,High School,Hometown,Ht.,No.,Position,Region,Wt.,Active Seasons,Misc,Bio,College
0,A.J. Haase,Bonne Terre,North County,"Bonne Terre, MO",6-4,NaN,TE,MO,245,2006-10,"{""Position:"": ""G"", ""Year:"": ""So."", ""Height:"": ...",2008: Started eight games at TE... LedYale end...,Yale
1,AJ Edwards,Seattle,South Kent School,"Seattle, Wash.",6-5,25,G,Wash.,190,2013-17,"{""Position:"": ""Pole Vault"", ""name"": ""Brendan S...",2013-14: Appeared in five games… Scored five p...,Yale
2,Aarica West,Buena Park,Vista Murrieta,"Buena Park, Calif.",5-10,31,G/F,Calif.,NaN,2009-12,"{""Hometown:"": ""Concord, MA"", ""High School:"": ""...",Before Yale: Played four years scholasticallyi...,Yale
3,Aaron Ault,Altamonte Springs,Lake Brantley,"Altamonte Springs, FL",6-3,93,LB,FL,194,2012-16,"{""Position:"": ""DB"", ""Year:"": ""So."", ""Height:"":...","2013: Played in all 10 games, mostly onspecial...",Yale
4,Aaron Bosgang,Port Washington,Paul D. Schreiber High School,"Port Washington, N.Y.",NaN,NaN,NaN,N.Y.,NaN,2016-17,"{""Position:"": ""OL"", ""Year:"": ""Jr."", ""Height:"":...",Off the Water: President of National Honor Soc...,Yale


In [516]:
indices = range(0, 7274)
stacked["Student_ID"] = indices

In [517]:
stacked = stacked.set_index(["Student_ID"])

In [518]:
stacked.shape

(7274, 13)

In [519]:
stacked.columns

Index([u'Name', u'City', u'High School', u'Hometown', u'Ht.', u'No.',
       u'Position', u'Region', u'Wt.', u'Active Seasons', u'Misc', u'Bio',
       u'College'],
      dtype='object')

In [520]:
# Split up the active seasons into start and end
stacked["StartSeason"] = stacked.apply(lambda x: int(x["Active Seasons"][0:4]), axis=1)
stacked["EndSeason"] = stacked.apply(lambda x: int(x["Active Seasons"][0:2] +
                                                   x["Active Seasons"][-2:]),
                                     axis=1)

In [521]:
stacked = stacked.drop("Active Seasons", axis=1)

In [522]:
stacked.head()

,Name,City,High School,Hometown,Ht.,No.,Position,Region,Wt.,Misc,Bio,College,StartSeason,EndSeason
Student_ID,,,,,,,,,,,,,,
0,A.J. Haase,Bonne Terre,North County,"Bonne Terre, MO",6-4,NaN,TE,MO,245,"{""Position:"": ""G"", ""Year:"": ""So."", ""Height:"": ...",2008: Started eight games at TE... LedYale end...,Yale,2006,2010
1,AJ Edwards,Seattle,South Kent School,"Seattle, Wash.",6-5,25,G,Wash.,190,"{""Position:"": ""Pole Vault"", ""name"": ""Brendan S...",2013-14: Appeared in five games… Scored five p...,Yale,2013,2017
2,Aarica West,Buena Park,Vista Murrieta,"Buena Park, Calif.",5-10,31,G/F,Calif.,NaN,"{""Hometown:"": ""Concord, MA"", ""High School:"": ""...",Before Yale: Played four years scholasticallyi...,Yale,2009,2012
3,Aaron Ault,Altamonte Springs,Lake Brantley,"Altamonte Springs, FL",6-3,93,LB,FL,194,"{""Position:"": ""DB"", ""Year:"": ""So."", ""Height:"":...","2013: Played in all 10 games, mostly onspecial...",Yale,2012,2016
4,Aaron Bosgang,Port Washington,Paul D. Schreiber High School,"Port Washington, N.Y.",NaN,NaN,NaN,N.Y.,NaN,"{""Position:"": ""OL"", ""Year:"": ""Jr."", ""Height:"":...",Off the Water: President of National Honor Soc...,Yale,2016,2017


In [523]:
# def strToDict(string):
#     if string:
#         string = string.encode('utf-8')
#         string = string.replace(" u'", " \"")
#         string = string.replace("': ", "\": ")
#         string = string.replace("', ", "\", ")
#         string = string.replace(" u\"", " \"")
#         string = string[0] + "\"" + string[3:]
#         string = string[:-2] + "\"" + string[-1]
#         try:
#             return json.loads(string, encoding="cp1252")
#         except:
#             print(string)
#             return {}
#     return {}

In [524]:
# stacked["Misc"] = stacked["Misc"].map(strToDict)

In [525]:
stacked.shape
print(json.loads(stacked["Misc"][104]))

None


In [530]:
def extractMajors(row):
    if row["Misc"] and not pd.isnull(row["Misc"]):
        misc = json.loads(row["Misc"])
        if misc and "Major:" in misc:
            return misc["Major:"]
    return ""

In [529]:
pd.isnull(stacked["Misc"][3347])

True

In [531]:
stacked["Major"] = stacked.apply(extractMajors, axis=1)

In [540]:
stacked.groupby("Major").count().sort_values("Name", ascending=False)

,Name,City,High School,Hometown,Ht.,No.,Position,Region,Wt.,Misc,Bio,College,StartSeason,EndSeason
Major,,,,,,,,,,,,,,
,5727,5727,5727,5727,3071,3129,3313,5727,2280,5722,5624,5727,5727,5727
Undeclared,557,557,557,557,336,257,275,557,183,557,536,557,557,557
Economics,212,212,212,212,116,94,103,212,81,212,202,212,212,212
Psychology,78,78,78,78,46,41,41,78,19,78,76,78,78,78
Political science,74,74,74,74,41,34,36,74,32,74,71,74,74,74
History,56,56,56,56,22,17,18,56,15,56,54,56,56,56
Undecided,46,46,46,46,25,23,24,46,21,46,45,46,46,46
Political Science,43,43,43,43,20,12,16,43,11,43,40,43,43,43
Government,31,31,31,31,15,17,18,31,11,31,31,31,31,31


In [541]:
# Seems about right.
processedPath = os.path.join(PROJ_ROOT, "data", "processed", "player_bios_processed.csv")
stacked.to_csv(processedPath, encoding='utf-8')